# `CofA` Daily_Launcher

### 1) create `logger`
### 2) create global variables
### 3) using `watchdog` wait for file to come into:
    - F:/Apps/CofA/

In [1]:
# IMPORT THE GOODIES
import os, sys, time
from time import sleep
from pathlib import Path
import fnmatch, glob, shutil
import wmi, psutil, subprocess
import tempfile, textwrap
from textwrap import wrap
from zipfile import ZipFile
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import email, smtplib, ssl
import os.path as op
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import subprocess, threading, logging
from threading import Timer
from PyPDF2 import PdfFileReader, PdfFileWriter
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
# GLOBAL VARIABLES
in_file = "C:/CofA/imp/Agilent_CofA_Letterhead_03-21-19.pdf"
#in_directory = "F:/APPS/CofA/"
#out_directory = "G:/C of A's/#Email Node/"
in_directory = "C:/Temp/F/APPS/CofA/"
staging_directory = ""
out_directory = "C:/Temp/G/C of A's/#Email Node/"
save_list = []
isEOD = False
time_start = pd.Timestamp.now()
df_save_list= pd.DataFrame()

**CLASSES**

In [2]:
class CofA_Event_Handler(FileSystemEventHandler): #{
    
    def __init__(self, save_dataframe, in_directory, out_directory): #{
        #self.save_list = save_list
        self.save_dataframe = save_dataframe
        self.in_directory = in_directory
        self.out_directory = out_directory
    #}
    
    """
    # DISPATCH ALWAYS CALLED FIRST JUST DESCRIBE EVENT
    def dispatch(self, event): #{
        event_str = str(event.event_type)
        event_path = Path(event.src_path)
        print("**********************")
        print("EVENT >>> " + event_str)
        print("SRC_PATH >>> " + str(event_path))
        print("**********************")
        # SEND OFF TO PERFORM ACTIONS BASED ON EVENT
        #perform_event_actions(the_event_type=event_str, the_event_path=event_path)
    #}
    """
    
    def on_created(self, event): #{
        global df_save_list
        # TRY THE FOLLOWING
        try: #{
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            ts = pd.Timestamp.now()  # CREATE TIME STAMP
            print("| CREATED >>> " + str(ts))
            # CREATE EVENT PATH VAR
            the_event_path = Path(event.src_path)  # WAS: the_event_path
            # CREATE 'file_name' VAR
            file_name = os.path.basename(the_event_path)
            # CHECK AND SEE IF FILE IS OF TYPE .PDF
            if fnmatch.fnmatch(file_name, "*.pdf"): #{
                # CREATE str TO HOLD FINAL COLUMN FOR TUPLE
                created_str = "Created"
                # CREATE EVENT LIST
                event_list = [str(file_name), str(ts), str(created_str)]
                df_save_list = append_to_dataframe(the_event_list=event_list,
                                    dataframe_to_append=df_save_list)
            #}
            else: #{
                print("NON-PDF CREATED AT " + str(ts))
            #}
            
        #}
        except: #{
            errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
            print("\n" + typeE +
                          "\n" + fileE +
                          "\n" + lineE +
                          "\n" + messageE)
        #}
        else: #{
            print("\t [Created-Event] FIN..")
        #}
        finally: #{
            # CREATE END-TIME VAR
            time_end = pd.Timestamp.now()
            # DETERMINE OVERALL RUN-TIME
            run_time = pd.Timedelta(time_end - time_start)
            # PRINT TOTAL RUNTIME
            print("\t [Created-Event] >>> time_alloted: " + str(run_time))
        #}
    #}
    
    def on_modified(self, event): #{
        global df_save_list
        # TRY THE FOLLOWING
        try: #{
            # CREATE INDEX FROM CURRENT DATAFRAME
            idx = pd.Index(data=df_save_list['Basename'], dtype=np.str)
            # CREATE EVENT PATH VAR
            the_event_path = Path(event.src_path)
            # CREATE 'file_name' VAR
            file_name = os.path.basename(the_event_path)
            """
            # CREATE FILE NAME CONV GENERATOR
            file_name_conv = generate_naming_convention(str(event.src_path))
            """
            # IF **MODIFIED** SRC_PATH IS ALREADY IN THE DATAFRAME...
            print("\n\t CHECKING IF FILE ADDED IS ALREADY IN DATAFRAME..." + str(file_name))
            if idx.contains(file_name): #{
                print(str(event.src_path) + " IS IN SAVE_DATAFRAME")
                # DONT ADD ANYTHING TO DATAFRAME
            #}
            # APPEND TO "df_save_list" WITH A MODIFIED TYPE
            else: #{
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                ts = pd.Timestamp.now()  # CREATE TIME STAMP
                print("| MODIFIED >>> " + str(ts))
                # CREATE EVENT PATH VAR
                the_event_path = Path(event.src_path)  # WAS: the_event_path
                # CREATE 'file_name' VAR
                file_name = os.path.basename(the_event_path)
                # CHECK AND SEE IF FILE IS OF TYPE .PDF
                if fnmatch.fnmatch(file_name, "*.pdf"): #{
                    # CREATE str TO HOLD FINAL COLUMN FOR TUPLE
                    modified_str = "Modified"
                    # CREATE EVENT LIST
                    event_list = [str(file_name), str(ts), str(modified_str)]
                    df_save_list = append_to_dataframe(the_event_list=event_list,
                                                       dataframe_to_append=df_save_list)
                #}
                else: #{
                    print("NON-PDF CREATED AT " + str(ts))
                #}
            #}
        #}
        except: #{
            errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
            print("\n" + typeE +
                          "\n" + fileE +
                          "\n" + lineE +
                          "\n" + messageE)
        #}
        else: #{
            print("\t [Modified-Event] VERY NICE GOOD JOB! ")
        #}
        finally: #{
            # CREATE END-TIME VAR
            time_end = pd.Timestamp.now()
            # DETERMINE OVERALL RUN-TIME
            run_time = pd.Timedelta(time_end - time_start)
            # PRINT TOTAL RUNTIME
            print("\t [Modified-Event] >>> time_alloted: " + str(run_time))
        #}
    #}
    
#}

In [3]:
class EmptyDataFrameException(Exception): #
    print("you suck!")
    pass
#}

you suck!


**FUNCTIONS**

In [4]:
"""
YOU KNOW WHAT THIS FUNCTION DOES
"""
def create_watermark(input_pdf, output, watermark): # {
    try:  # {
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)

        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()

        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()):  # {
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        # }

        with open(output, 'wb') as out:  # {
            pdf_writer.write(out)
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        print("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
        # }
    else: # {
        print("\t [watermark-pdf] FIN...")
    # }
    finally: # {
        # CREATE END-TIME VAR
        time_end = pd.Timestamp.now()
        # DETERMINE OVERALL RUN-TIME
        run_time = pd.Timedelta(time_end - time_start)
        # PRINT TOTAL RUNTIME
        print("\t [watermark-pdf] >>> time_alloted: " + str(run_time))
    # }
    return
# }

In [5]:
def get_all_file_paths(directory): #{
    
    # initializing empty file paths list 
    file_paths = [] 
    
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): #{
        for filename in files: #{
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath)
        #}
    #}
    
    # returning all file paths
    return file_paths
#}

In [6]:
"""
TAKES IN:
(1) path to pdf 
RETURNS: 
the STRING of that PDF to match the naming convention
in G:/C of A's/#Email Node/ from F:/APPS/CofA/
"""
def generate_naming_convention(the_pdf_path): #{
    # get/set filename to variable
    the_file_name = str(os.path.basename(the_pdf_path))
    # PERFORM STRING OPERATIONS
    #################################
    idx_mrk = the_file_name.rfind('@', 0, len(the_file_name))
    half1 = str(the_file_name[0:idx_mrk])
    half2 = str(the_file_name[idx_mrk + 1:len(the_file_name)])
    print("\t>>> HALF 1 == " + half1)
    print("\t>>> HALF 2 == " + half2)
    #  setup NEW FILE NAME (for copy)
    new_name = "part "
    new_name += str(half1)
    new_name += " CofA Lot# "
    new_name += str(half2)
    print("\t>>> NEW NAME == " + str(new_name))
    #################################
    return str(new_name)
#}

In [7]:
"""
TAKES IN: 
(1) LIST-type obj containing:
    - basename (*.pdf)
    - timestamp
    - "Created" **OR** "Modified category"
(2) DataFrame obj that the newest row will be appended too
RETURNS: DataFrame (not in-place) with newly
appended tuple row etc 
"""
def append_to_dataframe(the_event_list, dataframe_to_append): #{
    # TRY THE FOLLOWING
    try: #{
        # CHECK IF LIST
        if type(the_event_list) is list : #{  # WAS: (the_event_list is list)
            print("LIST check == PASS")
            # SEPERATE COLUMNS
            col_1 = str(the_event_list[0])
            col_2 = str(the_event_list[1])
            col_3 = str(the_event_list[2])
            # CREATE APPENDAGE FRAME
            df_appendage = pd.DataFrame(data=[the_event_list], 
                                        columns=['Basename', 'Timestamp', 'Created/Modified'])
            # CREATE INSTANCE OF DATAFRAME WE ARE RETURNING
            return_df = dataframe_to_append.append(df_appendage, ignore_index=True, sort=False)
            print(return_df.tail(5))
        #}
        else: #{
            print("LIST check == FAIL")
        #}
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        print("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else: #{
        print("\t [Append-2-DataFrame] FIN...")
        return return_df
    #}
    finally: #{
        # CREATE END-TIME VAR
        time_end = pd.Timestamp.now()
        # DETERMINE OVERALL RUN-TIME
        run_time = pd.Timedelta(time_end - time_start)
        # PRINT TOTAL RUNTIME
        print("\t [Append-2-DataFrame] >>> time_alloted: "+ str(run_time))
    #}
#}

In [8]:
def send_mail(send_from, send_to, subject, message, files=[], 
              server="cos.smtp.agilent.com", port=587, use_tls=True): #{
    print("SENDING MAIL... " + str(pd.Timestamp.now()))
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = COMMASPACE.join(send_to)
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject
    
    msg.attach(MIMEText(message))
    
    for path in files: #{
        part = MIMEBase('application', "octet-stream")
        with open(path, 'rb') as file: #{
            part.set_payload(file.read())
        #}
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                       'attachment; filename="{}"'.format(op.basename(path)))
        msg.attach(part)
    #}
    
    smtp = smtplib.SMTP(server, port)
    if use_tls: #{
        smtp.starttls()
    #}
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()
    return
#}

---

In [9]:
def execute_staging(): #{
    print("BEGIN>>>>>>>>>>>>>>>>>>>")
    # TRY THE FOLLOWING
    try: #{
        # re-declare GLOBAL VARS
        global in_file, in_directory, out_directory
        global df_save_list, to_email, from_email, isEOD
        print("[=================================================]")
        print("| EXECUTING STAGING >>> " + str(pd.Timestamp.now()) + "|")
        print("[=================================================]")
        print("| DIR >> " + str(in_directory))
        """
        # CREATE 'pdf_list' VAR FROM OUR SAVED DATAFRAME
        pdf_list = os.listdir(df_save_list)
        """
        print("| LENGTH OF SAVED_DATAFRAME: " + str(len(df_save_list)))
        ##############################################################
        # LOOP THRU AND WATERMARK / MOVE INTO G:/C of A's/#Email Node/
        # ** AND THEN EMAIL FOR EACH PDF OR CLUMP TOGETHER IN ZIP **
        if type(df_save_list) == DataFrame: #{
            # CREATE TEMP_DIR TO WORK INSIDE OF:
            with tempfile.TemporaryDirectory() as directory_name: #{
                # CHECK FOR EMPTY DATAFRAME, IF SO... BREAK.. ELSE...
                """
                if len(df_save_list) == 0: #{
                    raise EmptyDataFrameException
                #}
                """
                # CREATE PATH VAR and "Popen" via SUBPROCESS
                print("Opening Explorer Window...")
                the_dir = Path(directory_name)
                # open temp_folder in a new EXPLORER WINDOW
                subprocess.Popen('explorer ' + str(the_dir))
                # ITERATE THRU ROWS OF LISTS
                for row in df_save_list.itertuples(index=True, name="CofA"): #{
                    print("[=================================================]")
                    print("| New CoFA #" + str(int(row[0]) + 1) + ": " + str(row[1]))
                    # CREATE FILE_NAME VAR
                    file_name = str(row[1])
                    # CREATE FULL FILE_PATH VAR
                    file_path = os.path.join(in_directory, file_name)
                    print("| FILE-PATH: " + str(file_path))
                    print("[=================================================]")
                    ########## OLD INNER LOOP STARTED HERE< NOW ONLY 1 TEMP DIR
                    # print("| TEMP_DIRECTORY >>> " + str(the_dir))
                    # COPY NEWLY_MADE FILE INTO TEMP_DIR
                    shutil.copy(src=file_path, dst=the_dir)
                    # CREATE 'temp_path' FROM WITHIN TEMP_DOR
                    temp_path = os.path.join(the_dir, file_name)
                    # PERFORM STRING OPERATIONS / CREATE NAME_CONV
                    file_name_conv = generate_naming_convention(file_path)
                    # CREATE 'new_path' BY USING 'file_name_conv'
                    new_path = os.path.join(the_dir, file_name_conv)  # WAS: out_directory
                    create_watermark(input_pdf=temp_path,
                                     output=new_path,
                                     watermark=in_file)
                #}
                #sleep(5)
                # CREATE ZIPPED FOLDER OF COFA
                ##############################
                # CREATE NEW FOLDER WITH TODAYS DATE ON TITLE
                today_str = str(time_start)
                today_str = today_str[:10]
                # CREATE VARIABLE FOR NEW DIRECTORY PATH
                new_dir_path = os.path.join(the_dir, today_str)
                # CREATE NEW FOLDER FROM "new_path" VAR
                os.makedirs(new_dir_path)
                ####################################
                print("\n\n\n " + str(Path(the_dir) + "/part* \n\n\n"))
                # GLOB FOR WATERMARKED PDFS AND MOVE TO ZIP
                for name in sorted(glob.glob(str(Path(the_dir) + "part*"))): #{
                    print("\n" + str(name) + "\n")
                #}
                # path to folder which needs to be zipped 
                zip_directory = Path(new_dir_path)  # WAS: './python_files', THEN "the_dir"
                # COPY ENTIRE FILE TREE 
                # calling function to get all file paths in the directory 
                file_paths = get_all_file_paths(zip_directory) 
                # printing the list of all files to be zipped 
                print('Following files will be zipped:') 
                for f_name in file_paths: #{
                    print(f_name) 
                #}
                # CREATE PATH FOR ZIP FILE WE WISH TO CREATE
                zip_path = os.path.join(zip_directory, str(today_str + "-CofA.zip"))  # WAS: "my_python_files.zip"
                # writing files to a zipfile 
                with ZipFile(zip_path,'w') as zip: #{
                    # writing each file one by one 
                    for file in file_paths: #{
                        zip.write(file) 
                    #}
                #}
                print('All files zipped successfully!') 
                #########################
                break_input = input("Continue? Y\n")
            #}
            print("Name of temp_dir: " + str(the_dir))
            print("Directory exists after? " + str(the_dir.exists()))
            print("Contents after:" + str(list(the_dir.glob('*'))))           
        #}
        else: #{
            print("<<EXTREME ERROR>> (df_save_list NOT A DATAFRAME)")
            sys.exit(69)
        #}
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.info("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    # ELSE.... we complete without any errors
    else: #{
        # SET  BOOL TO TRUE SO WE BREAK WATCHDOG LOOP
        isEOD = True
    #}
    finally: #{
        # CREATE END-TIME VAR
        time_end = pd.Timestamp.now()
        # DETERMINE OVERALL RUN-TIME
        run_time = pd.Timedelta(time_end - time_start)
        # PRINT TOTAL RUNTIME
        print("\t [EXECUTE-STAGING] >>> time_alloted: " + str(run_time))
    #}
#}

---

**OVERWRITE `.txt` FILE WITH DATAFRME**

---

In [10]:
if __name__ == "__main__": #{
    # SETUP LOGGER
    try: #{
        logging.basicConfig(level=logging.INFO,
                           filename="C:/data/outbound/CofA_Daily_Launcher.log",
                           format='&(asctime)s:%(message)s',
                           datefmt='%Y-%d-%m-%H%M%S',
                           filemode='a')
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        print("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else: #{
        print("[Create-Logger] FIN...")
    #}
    ##########################################
    # RUN THIS CODE EVERY TIME NO MATTER WHAT
    # REGARDLESS.... TRY THE FOLLOWING....
    ########################################
    finally: #{
        ######################################
        # >>>>>>>> GLOBAL VARIABLES <<<<<<<< #
        ######################################
        in_file = "C:/CofA/imp/Agilent_CofA_Letterhead_03-21-19.pdf"
        #save_list = []
        df_save_list = pd.DataFrame(data=None, columns=['Basename', 'Timestamp', 'Created/Modified'], dtype=np.str)
        isEOD = False
        in_directory = "C:/Temp/F/APPS/CofA/"  #"F:/APPS/CofA/"
        out_directory =  "C:/Temp/G/C of A's/#Email Node/" #"G:/C of A's/#Email Node/"
        to_email = "agilent_cofa@agilent.com"
        from_email = "dere.bates@non.agilemt.com"
        time_start = pd.Timestamp.now()
    #}
    #global df_save_list, to_email, from_email
    print("[=================================================]")
    print("| SCANNING DIRECTORY >>> " + str(in_directory))
    print("| STORING IN DATAFRAME >>> " + str(df_save_list.head()))
    print("[=================================================]")
    print("| Initializing Time >>> " + str(pd.Timestamp.now()))
    print("[=================================================]")
    # CREATE TIMER VAR  // 12 HOURS... now 10.5 HOURS !! 08/07/2019
    # 37800
    t = Timer(30, execute_staging) # WAS 2592000,3600,720,600,43200,300 & 120
    # START TIMER
    t.start()
    # CREATE INSTANCE OF CUSTOM EVENT HANDLER
    event_handler = CofA_Event_Handler(df_save_list, in_directory, out_directory)
    observer = Observer()
    observer.schedule(event_handler=event_handler,
                      path=in_directory,
                      recursive=True)
    observer.start()
    # TRY THE FOLLOWING
    try: #{
        sleep_counter = 0
        while isEOD is False: #{
            # increase SLEEP COUNTEr & sleep...
            sleep_counter += 1
            sleep(1)
        #}
        else: #{
            print("[EOD] REACHED !!! <CLOSING SCRIPT>>")
            sys.exit(0)
        #}
    #}
    except KeyboardInterrupt: #{
        observer.stop()
    #}
    observer.join()
    #
#}

[Create-Logger] FIN...
[=================================================]
| SCANNING DIRECTORY >>> C:/Temp/F/APPS/CofA/
| STORING IN DATAFRAME >>> Empty DataFrame
Columns: [Basename, Timestamp, Created/Modified]
Index: []
[=================================================]
| Initializing Time >>> 2019-08-14 16:05:30.492357
[=================================================]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
| CREATED >>> 2019-08-14 16:05:31.234414
LIST check == PASS
                 Basename                   Timestamp Created/Modified
0  ICP-079@0006479330.pdf  2019-08-14 16:05:31.234414          Created
	 [Append-2-DataFrame] FIN...
	 [Append-2-DataFrame] >>> time_alloted: 0 days 00:00:00.748061
	 [Created-Event] FIN..
	 [Created-Event] >>> time_alloted: 0 days 00:00:00.748561
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
| CREATED >>> 2019-08-14 16:05:32.689588
LIST check == PASS
                 Basename                   Timestamp Created/Modified
0  ICP-079@0006479330.pdf 

In [11]:
%tb

No traceback available to show.
